In [2]:
import requests
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import csv
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from nltk import pos_tag
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [3]:
##SCRAPER CODE
# names = []
# reviews = []
# grades = []
# url = 'https://www.temptalia.com/category/reviews/page/1035/'
# response = requests.get(url)
# page = response.text
# soup = BeautifulSoup(page, "lxml")
# product_links = []
# links = soup.find_all(class_='btn btn-secondary btn-lg more-link')
# for link in links:
#     product_links.append(link.get('href'))
# for link in product_links:
#     response = requests.get(link)
#     soup = BeautifulSoup(response.text, "lxml")
#     product_names = soup.find_all(class_ = 'hidden-md-up text-xs-center text-md-left text-uppercase pt-1 mb-2')
#     product_reviews = (soup.find_all('p'))
#     product_grades = (soup.find_all(class_ = 'glossover-grade f-7 bold'))
#     counter = 0
#     for review in product_reviews:
#         if '$' in review.text and len(review.text) > 35:
#             reviews.append(review.text)
#             names.append(product_names[counter].text)
#             grades.append(product_grades[counter].text)
#             counter += 1
# new_df = pd.DataFrame(data = {'names': names, 'reviews': reviews, 'grades': grades})
# new_df.to_csv('temp_sheet.csv')
# new_df = new_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# new_df.tail()

In [3]:
new_df = pd.read_csv('output.csv', encoding = "ISO-8859-1")

In [4]:
new_df.tail()

,Brand,Name,Product,Review,Grade
5168,MAC,OVERSPICED,Lipgloss,"a dirty muted coral. ItÕs a warm-toned, light-...",C+
5169,MAYBELLINE,NUDE ILLUSION (060),Lipgloss,"light-medium peachy beige with soft, warm unde...",B
5170,SURRATT BEAUTY,AU NATURELLE,Lipgloss,a universal neutral with a kiss of gold. ItÕs ...,C+
5171,CHANEL,SENSUEL (11),Lipgloss,a peachy brown with a very fine gold shimmer. ...,B
5172,NARS,BELLE DE JOUR,Lipstick,"a light-medium, yellowed peach with warm under...",D


In [5]:
def removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

In [6]:
clean = new_df['Review'].apply(removeNonAscii)

In [7]:
new_df['Review'] = clean

In [8]:
new_df.replace('-', ' ', regex=True, inplace=True)

In [9]:
new_df.sample(n=20)

,Brand,Name,Product,Review,Grade
1046,SEPHORA,COPPER RUSH (291),Eyeshadow,"a deep copper with warm, brownish undertones a...",A
640,DOSE OF COLORS,AESTHETIC,Lipstick,"a rich, medium dark copper with warm, red unde...",A
2816,SMASHBOX,CRUSH IT,Lipstick,"a deep plum with subtle, cool undertones and a...",A
655,DOSE OF COLORS,SOL MATE,Highlighter,a medium dark gold with warm undertones and a ...,A
732,OFRA,BLIND THE HATERS,Highlighter,"a rich, medium dark copper with warm, brown un...",A
4126,LORAC,BLACK,Palette,a mix of darker neutral shades that ranges fro...,B
28,COLOUR POP,KUMQUAT,Lipgloss,"a pale, bluish white with cool undertones and ...",B
2706,TOM FORD BEAUTY,GOLD DUST,Bronzer,"a muted, medium dark brown with warm, red unde...",A
2625,KAT VON D,DREAMER,Lipstick,"a bright, light medium cyan blue with cooler u...",B
4584,MAC,A PASSIONATE QUEST,Palette,a mix of cool pink and purple. Three of the fo...,A


In [10]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', 
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)
cv_data = count_vectorizer.fit_transform(new_df['Review'])
lsa_ = TruncatedSVD(n_components=50)
lsa_data = lsa_.fit_transform(cv_data)

In [78]:
new_vec = lsa_.transform(count_vectorizer.transform(['a brighter, medium purple with soft, warm pink undertones and a frosted sheen. It had rich color coverage that applied well to bare skin with smooth, even color. The texture was lightly creamy, dense but not stiff or difficult to pick up with a dry brush. It blended out easily along the edges and lasted well for eight and a half hours on my lid.']))
nn = NearestNeighbors(n_neighbors=1, metric='cosine')
nn.fit(lsa_data)
results = nn.kneighbors(new_vec, n_neighbors=5)


In [79]:
for i in range(len(results[1][0])):
        print(new_df.iloc[results[1][0][i]])

Brand                                                  NABLA
Name                                            DAPHNE NO. 2
Product                                            Eyeshadow
Review     a medium plum with warm, brown undertones and ...
Grade                                                     A 
Name: 764, dtype: object
Brand                                                   NARS
Name                                                   DOURO
Product                                            Eyeshadow
Review     a bright, medium yellow with a mostly matte fi...
Grade                                                     B+
Name: 404, dtype: object
Brand                                                    MAC
Name                                                  MARGIN
Product                                                Blush
Review     a soft, peachy copper with warm undertones and...
Grade                                                     A 
Name: 798, dtype: object
Brand     

In [84]:
def get_nearest_prods():
    new_vec = lsa_.transform(count_vectorizer.transform([input()]))
    nn = NearestNeighbors(metric='cosine')
    nn.fit(lsa_data)
    results = nn.kneighbors(new_vec, n_neighbors=5)
    rec_strings = []
    for i in range(len(results[1][0])):
        rec_strings.append(f"Try {new_df.iloc[results[1][0][i]]['Name']}, a {new_df.iloc[results[1][0][i]]['Product']} by {new_df.iloc[results[1][0][i]]['Brand']}.")
    return rec_strings

    

In [85]:
get_nearest_prods()

a non sticky gloss


['Try THE MOST, a Lipgloss by DOSE OF COLORS .',
 'Try OVER THE TOP, a Lipstick by DOSE OF COLORS.',
 'Try DUA LIPA, a Lipgloss by MAC.',
 'Try ILUVSARAHII 2, a Palette by COLOUR POP.',
 'Try FAUX PAS, a Lipgloss by SURRATT BEAUTY.']